# NHSBSA English Prescribing Data (EPD) Analysis

## Part 2: SQL, AWS EC2 Instance and Exploratory Data Analysis

### SQL

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import requests

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = "https://opendata.nhsbsa.net/api/3/action/datastore_search_sql?resource_id=EPD_201401&sql=SELECT * from `EPD_201401` limit 5"
pd.DataFrame(requests.get(url).json()['result']['result']['records']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   BNF_CODE                      5 non-null      object 
 1   TOTAL_QUANTITY                5 non-null      float64
 2   POSTCODE                      5 non-null      object 
 3   YEAR_MONTH                    5 non-null      int64  
 4   UNIDENTIFIED                  5 non-null      bool   
 5   PRACTICE_NAME                 5 non-null      object 
 6   BNF_CHAPTER_PLUS_CODE         5 non-null      object 
 7   ACTUAL_COST                   5 non-null      float64
 8   QUANTITY                      5 non-null      float64
 9   REGIONAL_OFFICE_CODE          5 non-null      object 
 10  ITEMS                         5 non-null      int64  
 11  ADDRESS_4                     5 non-null      object 
 12  AREA_TEAM_CODE                5 non-null      object 
 13  ADDRESS_2

In [3]:
# Define the parameters for the SQL query

years = [19,20,21]
months = list(range(13))[1:]
resources = []
for year in years:
    resource_name_year = "EPD_20"+str(year)
    for month in months:
        resource_names = resource_name_year+str(month).zfill(2)
        resources.append(resource_names)

        
EPDs_needed = resources[:30]

In [4]:
# Define the url for the API call
base_endpoint = "https://opendata.nhsbsa.net/api/3/action"
action_method = "/datastore_search_sql?" # SQL 

In [5]:
"""
The filters used in the Pandas dataframe within part 1 will inform the SQL API query.

CHEMICAL_SUBSTANCE_BNF_DESCR = 'Cetirizine hydrochloride'                           
CHEMICAL_SUBSTANCE_BNF_DESCR = 'Loratadine'                                       
CHEMICAL_SUBSTANCE_BNF_DESCR = 'Desloratadine'                                        
CHEMICAL_SUBSTANCE_BNF_DESCR = 'Fexofenadine hydrochloride'
CHEMICAL_SUBSTANCE_BNF_DESCR = 'Acrivastine'                                           
CHEMICAL_SUBSTANCE_BNF_DESCR ='Bilastine'                                             
CHEMICAL_SUBSTANCE_BNF_DESCR ='Levocetirizine'                               
CHEMICAL_SUBSTANCE_BNF_DESCR ='Mizolastine'                                   
CHEMICAL_SUBSTANCE_BNF_DESCR ='Chlorphenamine maleate'                               
BNF_DESCRIPTION != 'Chlorphenamine 10mg/1ml solution for injection ampoules'            
CHEMICAL_SUBSTANCE_BNF_DESCR='Promethazine hydrochloride'
BNF_DESCRIPTION != 'Promethazine 25mg/1ml solution for injection ampoules'             
BNF_DESCRIPTION !='Phenergan 25mg/1ml solution for injection ampoules'                


"""

"\nThe filters used in the Pandas dataframe within part 1 will inform the SQL API query.\n\nCHEMICAL_SUBSTANCE_BNF_DESCR = 'Cetirizine hydrochloride'                           \nCHEMICAL_SUBSTANCE_BNF_DESCR = 'Loratadine'                                       \nCHEMICAL_SUBSTANCE_BNF_DESCR = 'Desloratadine'                                        \nCHEMICAL_SUBSTANCE_BNF_DESCR = 'Fexofenadine hydrochloride'\nCHEMICAL_SUBSTANCE_BNF_DESCR = 'Acrivastine'                                           \nCHEMICAL_SUBSTANCE_BNF_DESCR ='Bilastine'                                             \nCHEMICAL_SUBSTANCE_BNF_DESCR ='Levocetirizine'                               \nCHEMICAL_SUBSTANCE_BNF_DESCR ='Mizolastine'                                   \nCHEMICAL_SUBSTANCE_BNF_DESCR ='Chlorphenamine maleate'                               \nBNF_DESCRIPTION != 'Chlorphenamine 10mg/1ml solution for injection ampoules'            \nCHEMICAL_SUBSTANCE_BNF_DESCR='Promethazine hydrochloride'\nBNF_DESCRIPTION !

In [109]:
resource_name = "EPD_202002"

In [115]:
query = f"""
    SELECT 
    
    TOTAL_QUANTITY, 
    POSTCODE, 
    YEAR_MONTH,
    PRACTICE_NAME,
    ACTUAL_COST,
    BNF_CHEMICAL_SUBSTANCE,
    REGIONAL_OFFICE_NAME,
    CHEMICAL_SUBSTANCE_BNF_DESCR,
    AREA_TEAM_NAME
    
    FROM 
        `{resource_name}`     
    WHERE
    1=1
    AND CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Cetirizine hydrochloride'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Loratadine'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Fexofenadine hydrochloride'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Acrivastine'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Bilastine'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Levocetirizine'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Mizolastine'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Chlorphenamine maleate'}'
        OR CHEMICAL_SUBSTANCE_BNF_DESCR = '{'Promethazine hydrochloride'}'
       
"""

In [116]:
# Send API call and grab the response as a json
response = requests.get(
    url=(
        base_endpoint 
        + action_method 
        + "resource_id=" + resource_name
        + "&"
        + "sql=" + requests.utils.quote(query) # Encode url
    ),
    verify=False
).json()

In [117]:
response.keys()

dict_keys(['help', 'success', 'result'])

In [120]:
response['result']

{'records_truncated': 'true',
 'gc_urls': [{'url': 'https://storage.googleapis.com/dx-nhs-tmp/anon0737d9acc1e411ef08afe0463c17a975734863ef/`EPD_202002`-000000000000.csv.gz'}],
 'help': 'https://demo.ckan.org/api/3/action/help_show?name=datastore_search_sql',
 'success': 'true'}

In [140]:
Met_Pollen_Data = ['PollenStationData2011.xlsx',
                   'PollenStationData2012.xlsx', 
                   'PollenStationData2013.xlsx', 
                   'PollenStationData2014.xlsx', 
                   'PollenStationData2015.xlsx', 
                   'PollenStationData2016.xlsx',
                   'PollenStationData2017.xlsx', 
                   'PollenStationData2018.xlsx', 
                   'PollenStationData2019.xlsx',
                   'PollenStationData2020.xlsx', 
                   'PollenStationData2021vs30June.xlsx']

sheets_required = ['Bath',
                   'Belfast',
                   'Cambridge',
                   'Cardiff',
                   'East Riding',
                   'Edinburgh',
                   'Eskdalemuir',
                   'Exeter',
                   'Hull',
                   'Invergowrie',
                   'Ipswich',
                   'Isle of Wight',
                   'Leicester',
                   'London, Islington',
                   'London, Kings College',
                   'Plymouth',
                   'Rotherham',
                   'Worcester',
                   'York']

In [158]:
df = pd.read_excel('PollenStationData2011.xlsx', sheet_name =sheets_required)
df_new = pd.concat(df)
df_new.head()

Date Cory  Alnu  Sali  Betu  Frax  Ulmu  Quer  Plat  Poac  Urti  \
Bath 0 2011-05-16   -1  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0   
     1 2011-05-17   -1  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   0.0  -1.0   
     2 2011-05-18   -1  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   1.0  -1.0   
     3 2011-05-19   -1  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   7.0  -1.0   
     4 2011-05-20   -1  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0   4.0  -1.0   

        Arte  Ambr  
Bath 0  -1.0  -1.0  
     1  -1.0  -1.0  
     2  -1.0  -1.0  
     3  -1.0  -1.0  
     4  -1.0  -1.0

https://stackoverflow.com/questions/15159253/python-try-except-block-does-not-recognize-error-type

In [129]:
Pollen_types_ = {'Cory':'Hazel Pollen',
                'Alnu': 'Alder Pollen',
                'Sali':'Willow Pollen',
                'Betu': 'Birch Pollen',
                'Frax': 'Ash Pollen',
                'Ulmu':'Elm Pollen',
                'Quer':'Oak Pollen',
                'Plat':'Plane (tree) Pollen',
                'Poac':'Grass Pollen',
                'Urti':'Nettle Pollen',
                'Arte':'Mugwort Pollen',
                'Ambr': 'Ragweed Pollen'}


from xlrd import XLRDError

dfs=[]


for pollen_data in Met_Pollen_Data:
    try:
        df = pd.read_excel(pollen_data, sheet_name =sheets_required)
    
    except XLRDError:
        pass
        
    
    for location in list(df.keys()):
            df[location]['Location']= location
                    
    df_new = pd.concat(df)
    
    df_new.reset_index(drop=True, inplace = True)
    
    dfs.append(df_new)
    met_pollen_df =pd.concat(dfs)

met_pollen_df.rename(mapper=Pollen_types, axis=1, inplace=True)

In [157]:
met_pollen_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26664 entries, 0 to 2423
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 26664 non-null  datetime64[ns]
 1   Hazel Pollen         25773 non-null  object        
 2   Alder Pollen         25729 non-null  float64       
 3   Willow Pollen        25729 non-null  float64       
 4   Birch Pollen         25729 non-null  float64       
 5   Ash Pollen           25729 non-null  float64       
 6   Elm Pollen           25729 non-null  float64       
 7   Oak Pollen           25729 non-null  float64       
 8   Plame (tree) Pollen  25729 non-null  float64       
 9   Grass Pollen         25729 non-null  float64       
 10  Nettle Pollen        25729 non-null  float64       
 11  Mugwort Pollen       25729 non-null  float64       
 12  Ragweed Pollen       25729 non-null  float64       
 13  Location             26664 non-n

### AWS Cloud Computing

In [ ]:
EPD_Server = xmlrpc.client.ServerProxy('http://54.196.229.110:2021')
df = EPD_Server.LargeCSVsChop

In [ ]:
Jan_2020_df = LargeCSVsChop(datasets[72:73])
Feb_2020_df = LargeCSVsChop(datasets[73:74])
Mar_2020_df = LargeCSVsChop(datasets[74:75])
Apr_2020_df = LargeCSVsChop(datasets[75:76])
May_2020_df = LargeCSVsChop(datasets[76:77])
Jun_2020_df = LargeCSVsChop(datasets[77:78])
Jul_2020_df = LargeCSVsChop(datasets[78:79])
Aug_2020_df = LargeCSVsChop(datasets[79:80])
Sep_2020_df = LargeCSVsChop(datasets[80:81])
Oct_2020_df = LargeCSVsChop(datasets[81:82])
Nov_2020_df = LargeCSVsChop(datasets[82:83])
Dec_2020_df = LargeCSVsChop(datasets[83:84])

HayFev_df_2020_list = [Jan_2020_df,Feb_2020_df ,Mar_2020_df,Apr_2020_df,May_2020_df,Jun_2020_df,Jul_2020_df,
                       Aug_2020_df,Sep_2020_df,Oct_2020_df,Nov_2020_df,Dec_2020_df ]

HayFev_df_2020 = pd.concat(HayFev_df_2020_list)

HayFev_df_2020.to_csv('HayFev_df_2020.csv') 

Given the cost of hardware, I am more inclined to look for workarounds such as these to facilitate and speed up personal projects. I hope this has been useful to others and I would love comments and feedback on my code. 



### Whole year of Data

Each csv is read in individual lines as opposed to one to allow for fleixibility when the kernel needs to be interrupted due to the time taken or any other issue.

In [ ]:
Feb_2019_df = LargeCSVsChop(datasets[61:62])
Mar_2019_df = LargeCSVsChop(datasets[62:63])
Apr_2019_df = LargeCSVsChop(datasets[63:64])
May_2019_df = LargeCSVsChop(datasets[64:65])
Jun_2019_df = LargeCSVsChop(datasets[65:66])
Jul_2019_df = LargeCSVsChop(datasets[66:67])
Aug_2019_df = LargeCSVsChop(datasets[67:68])
Sep_2019_df = LargeCSVsChop(datasets[68:69])
Oct_2019_df = LargeCSVsChop(datasets[69:70])
Nov_2019_df = LargeCSVsChop(datasets[70:71])
Dec_2019_df = LargeCSVsChop(datasets[71:72])

HayFev_df_2019_list = [Jan_2019_df,Feb_2019_df ,Mar_2019_df,Apr_2019_df,May_2019_df,Jun_2019_df,Jul_2019_df,
                       Aug_2019_df,Sep_2019_df,Oct_2019_df,Nov_2019_df,Dec_2019_df ]

HayFev_df_2019 = pd.concat(HayFev_df_2019_list)

HayFev_df_2019.to_csv('HayFev_df_2019.csv') 